In [1]:
import cadquery as cq
from cadquery import exporters
from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, Faces, Vertices, show, 
                                       replay, enable_replay, disable_replay, reset_replay)
from jupyter_cadquery import set_sidecar, set_defaults

from math import sin, radians, sqrt

import ipywidgets as widgets

Overwriting auto display for cadquery Workplane and Shape


In [2]:
set_defaults(axes=True, grid=True, axes0=True, timeit=False, cad_width=640, height=800)

set_sidecar("Z", init=True)
replay_box = widgets.Checkbox(False, description='Enable Replay')
out = widgets.Output()
display(replay_box)
display(out)

@out.capture()
def changed(e):
    if e["new"]:
        enable_replay(warning=False)
    else:
        disable_replay()

replay_box.observe(changed, names='value')

Checkbox(value=False, description='Enable Replay')

Output()

In [8]:
key_sizes = [10, 8, 6, 5, 4, 3.5, 3, 2.5, 2, 1.5]
key_sizes = map((lambda x: x+0.02), key_sizes)

thickness = 2

def key_path(size, length):
    return (
        cq.Workplane("XZ")
        .line(0, length-size)
        .radiusArc((size, length), size)
        .line(length-size, 0)
    )

replay(key_path(5, 20))

[10.02, 8.02, 6.02, 5.02, 4.02, 3.52, 3.02, 2.52, 2.02, 1.52]
Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'
Done, using side car 'Z'


In [4]:
def hex_d(d):
    return d/sqrt(3)*2

def key_void(size, height):
    return (
        cq.Workplane("XY")
        .sketch()
        .regularPolygon(hex_d(size)/2, 6)
        .finalize()
        .sweep(key_path(size, height))
        .workplane(offset = -height, invert=True)
        .sketch()
        .regularPolygon(hex_d(size)/2, 6)
        .finalize()
        .extrude("next")
        .faces(">Z")
        .workplane(invert=True)
        .rect(height, hex_d(size)-0.000001, centered=(False, True))
        .extrude("next")
    )

replay(key_void(4, 10))

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'
Done, using side car 'Z'


In [5]:
result = (
    cq.Workplane("XY")
)

for size_x in key_sizes:
    size_y = hex_d(size_x)
    width = size_y+thickness*2
    height = 3*size_x+thickness
    result = result.box(width, width, height, centered=False).center(0, width)
    
result = result.end()#.edges("|Y").chamfer(1)
offset = 0
for size_x in key_sizes:
    height = 3*size_x+thickness
    size_y = hex_d(size_x)
    offset += thickness+size_y/2
    #if size_x == 10:
        # magic to fix the StdFail_NotDone: BRep_API: command not done
        #size_x -= 0.0000001
    result = result.cut(key_void(size_x, height-size_x).translate((thickness+size_x/2, offset, 0)))
    offset += size_y/2+thickness

replay(result)

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'
Done, using side car 'Z'


In [6]:
exporters.export(result, '/tmp/x.stl')